# Support Vector Machines

## Project Setup

### Importing the Libraries 

In [1]:
# Scikit-learn utilities
import numpy as np
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

### Importing the Dataset

In [2]:
import sys
import os

# To allow for importing of 'utils' module from parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

from utils import import_data

dataset_path = "../data"
dataset_version = "automated-v1.0"

automated_dataset = import_data(dataset_path, dataset_version)